In [3]:
#score_list = [[]for i in range(2)]

def just_run(move_number):
    
    global joints, coord, Texts, score_list, DJoints
    import pandas as pd 
    from imutils.video import WebcamVideoStream
    import imutils
    import matplotlib.pyplot as plt
    import cv2
    import numpy as np
    import threading
    import mediapipe as mp
    from matplotlib import pyplot as plt
    import time
    from imutils.video import FPS
    import math

    mp.solutions.drawing_utils._VISIBILITY_THRESHOLD = 0.01
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_pose = mp.solutions.pose

    MANY_KP = [1,2, 6, 15, 16, 19, 20, 21, 22, 26, 32, 35, 37]
    
    z_scale_factor = np.array([ 1, 
                               1, 1, 1, 1, 1, 
                               1, 1, 1, 1, 1, 
                               0.2, 0.2, 0.2, 0.2, 0.2, 
                               0.2, 1, 1, 1, 1, 
                               1, 1, 0.5, 0.5, 0.5, 
                               0.5, 0.5, 0.5, 1, 1, 
                               0.5, 0.5])


    #csv 파일 읽어오기
    df_keypose = pd.read_csv('combine.csv')
    df_keypose['Name'].unique()
    


    #동작 정하기
    target_motion = df_keypose['Name'].unique()[int(move_number)]

    df_target = df_keypose[df_keypose['Name'] == target_motion]
    
    motion_name = df_target['Move'][int(move_number)]
    print('<<',move_number,'.',motion_name,'>>\n')

    #동작의 조인트 값들 리스트
    Joints = df_target['Joints'][int(move_number)]
    #정면인지 측면인지 확인을 위한 리스트
    Coord = df_target['Coord'][int(move_number)]
    #동작 타겟 각도 리스트
    Angles = df_target['Angles'][int(move_number)]
    
    #타겟 동작 실행 시간 ㄹㅣ스트
    #target_time = df_target['Time'][int(move_number)].split(',')
    #target_times = list(map(float,target_time))
    
    #거리 구해야하는 joint
    DJoints =  df_target['Distance'][int(move_number)]
    if str(DJoints) != "nan":
        DJoints = eval(DJoints)
    else:
        DJoints = None
    def angle_differ(arr):
        angle_arr = []
        for i in range(len(arr)-1):
            appends = np.abs(arr[i+1] - arr[i])
            appends = appends.tolist()
            angle_arr.append(appends)
        angle_arr = np.array(angle_arr)
        return angle_arr

    def make_joint(Joints):
        joint = eval(Joints)
        return joint


    if move_number+1 in MANY_KP:
    
        ## 시퀀스 동작일 때 필요한 값들
        joints = make_joint(Joints)
        j_len_list = []
        for i in range(len(joints)):
            if i > 0:
                j_len_list.append(j_len_list[i-1]+len(joints[i]))
            else:
                j_len_list.append(len(joints[i]))

        coord = make_joint(Coord)
        target_angles = make_joint(Angles)
        
        #동작 threshold 값
        #target_thre = [int(df_target['Threshold'][move_number]) for i in range(len(target_angles))]
        target_thre = [int(5) for i in range(len(target_angles))]
        keypose_check = False
        
    else :
        ## 단일 동작일 때 필요한 값들
        joints = make_joint(Joints)
        coord = make_joint(Coord)
        target_angles = make_joint(Angles)
        
        #동작 threshold 값
        #target_thre = [int(df_target['Threshold']) for i in range(len(target_angles))]
        target_thre = [int(5) for i in range(len(target_angles))]
        keypose_check = True

            
    def get_angle(a, b, c):
        ba = a - b
        bc = c - b

        ba[2] = 0
        bc[2] = 0

        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle)

        return np.degrees(angle)

    def get_stand(a, b):
        ba = a - b
        x_differ = ba[0]
        y_differ = ba[1]
        z_differ = ba[2]
        return [x_differ,y_differ,z_differ]

    def get_gradient(a,b):
        c = [b[0],a[1],a[2]]

        ab = b - a
        ac = c - a
        ab[2] = 0
        ac[2] = 0
        cosine_angle = np.dot(ab, ac) / (np.linalg.norm(ab) * np.linalg.norm(ac))
        angle = np.arccos(cosine_angle)

        return np.degrees(angle)

    def get_distance(a,b):
        x = a[0]-b[0]
        y = a[1]-b[1]
        return round(math.sqrt((x * x) + (y * y)),2)

    ##시퀀스 적은 동작
    def __joint_angle(data):
        global joints,coord

        return np.array([ get_angle( *data[j, :3]) if len(j) == 3 else get_gradient( *data[j, :3]) for j in joints]),np.array([ get_stand( *data[j, :3]) for j in coord])
      
    ## 시퀀스 많은 동작
    def get_joint_angle(data):
        global joints,coord
        

        return np.array([ get_angle( *data[j, :3]) if len(j) == 3 else get_gradient( *data[j, :3]) for k in joints for j in k]),np.array([ get_stand( *data[j, :3]) for j in coord])


    
    # 디스플레이 부분
    # Display function
    # ================
    
    
    
    
    ## 키포즈 한개 main
    def view_1():
        #global score_list
        global joints
        
        print('원본 스트레칭 영상 각도')
        for i in range(len(target_angles)):
            if len(target_angles) ==2 and i ==0:
                print('방향1')
            elif i == 1:
                print('방향2')
            for j,a in zip(joints,target_angles[i]):
                
                print(j,':',a,'°')
            print('\n')
        
        cap = WebcamVideoStream(src=0).start()
        #cap = WebcamVideoStream(src = '../Desktop/3.mp4').start()

        target_prev_time = time.time()
        coords = np.zeros((1, 38,4))
        
        target_prev_time = time.time()
        
        cur_id = 0
        sec = False
        second = 0
        correct_second = 0

        i = 0
        cnt = 0
        
        #score_list = [[]for i in range(len(target_angles))]
        
        with mp_pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            enable_segmentation=False) as pose:


            while True:
                
                frame = cap.read()
                frame = imutils.resize(frame, width=640, height=480)

                image_height, image_width, _ = frame.shape
                results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


                #frame = cv2.flip(frame, 1) # if your camera reverses your image

                if results.pose_landmarks is None:
                    print('nope')
                    continue
                    


                cur_coord = np.array([(results.pose_landmarks.landmark[i].x, 
                                  results.pose_landmarks.landmark[i].y, 
                                  results.pose_landmarks.landmark[i].z, 
                                  results.pose_landmarks.landmark[i].visibility) for i in range(33)])#if results.pose_landmarks.landmark[i].visibility > 0.2 else (0,0,0,0) for i in range(33) ])


                cur_coord[:, 2] = cur_coord[:, 2] * z_scale_factor

                cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[31:32, :2], cur_coord[27:28, 2:], axis=0), (-1,4)), axis= 0)
                cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[32:33, :2], cur_coord[28:29, 2:], axis=0), (-1,4)), axis= 0)
                cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[7:8, :2], cur_coord[8:9, 2:], axis=0), (-1,4)), axis= 0)
                cur_coord = np.append( cur_coord, np.reshape([ cur_coord[8, 0], cur_coord[7, 1], cur_coord[8, 2], cur_coord[8, 3]], (-1,4)), axis= 0)
                cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[0:1, :2], cur_coord[8:9, 2:], axis=0), (-1,4)), axis= 0)

                coords = np.append(coords[-2:], np.reshape(cur_coord, (-1,38,4)), axis=0)

                coord = np.median(coords, axis =0 )
                
                
                #디렉스에서 요청했던 1, 2 값 추가 39번 : 1, 40 번 : 2
                shoulder_m = [x+y//2 for x,y in zip(coord[11], coord[12])]
                gol_m = [x+y//2 for x,y in zip(coord[23], coord[24])]
                
                coord = np.append(coord, [shoulder_m], axis=0)
                coord = np.append(coord, [gol_m], axis=0)
                
                
                input_angles, input_stands = __joint_angle(coord)


                #frame = cv2.resize(frame, dsize=(640, 480 ))
                
        
                
                cv2.rectangle(frame, (0, 0), (230, (len(joints)-1)*30+85), (255,255,255), -1)
                joint_title = "User Inputs"
                cv2.putText(frame, joint_title, (0, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 120, 105), 2)
                
                z = 75
                for i in range(len(input_angles)):
                    text = f"{joints[i]} : {input_angles[i]:.0f}"
                    cv2.putText(frame, text, (10, z), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,0.7, (0, 0, 0), 2)
                    z = z + 30
                
                cv2.imshow('frame', frame)
                
                key = cv2.waitKey(1) & 0xFF

                if key == ord("q"):
                    break

    ## 키포즈 여러개 main
    def view_2():
        #global s_list
        global joints, cur_id, DJoints
        
        print('원본 스트레칭 영상 각도')
        for i in range(len(target_angles)):
            print(i,'번째 phase')
            for j,a in zip(joints[i],target_angles[i]):
                if (DJoints) != None and j == DJoints[0]:
                    print(j,':',a)
                else:
                    print(j,':',a,'°')
            
            
        def getphase():
            global cur_id
            while True:
                print('\n 총 Phase : 0 ~',len(target_angles)-1 )
                input_id = int(input('원하는 페이즈 번호를 입력하세요 : '))
                
                if input_id > len(target_angles)-1:
                    print('다시 입력해주세요.')
                else:
                    print('yes')
                    cur_id = input_id  
                
        if keypose_check == False:
            # 데몬 쓰레드
            t1 = threading.Thread(target=getphase)
            t1.daemon = True
            t1.start()
            
        #cap = WebcamVideoStream(src = '../Desktop/37.mp4').start()
        cap = WebcamVideoStream(src=0).start()
        
        target_prev_time = time.time()
        coords = np.zeros((1, 38,4))
        
        target_prev_time = time.time()
            
        cur_id = 0
        sec = False
        second = 0
        correct_second = 0 
        i = 0
        cnt = 0
        
        #s_list = [[]for i in range(len(target_angles))]
        
        
            
        with mp_pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            enable_segmentation=False) as pose:


            while True:
                
                frame = cap.read()
                frame = imutils.resize(frame, width=640, height=480)

                image_height, image_width, _ = frame.shape
                results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


                #frame = cv2.flip(frame, 1) # if your camera reverses your image

                if results.pose_landmarks is None:
                    print('nope')
                    continue
                    


                cur_coord = np.array([(results.pose_landmarks.landmark[i].x, 
                                  results.pose_landmarks.landmark[i].y, 
                                  results.pose_landmarks.landmark[i].z, 
                                  results.pose_landmarks.landmark[i].visibility) for i in range(33)])#if results.pose_landmarks.landmark[i].visibility > 0.2 else (0,0,0,0) for i in range(33) ])


                cur_coord[:, 2] = cur_coord[:, 2] * z_scale_factor

                cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[31:32, :2], cur_coord[27:28, 2:], axis=0), (-1,4)), axis= 0)
                cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[32:33, :2], cur_coord[28:29, 2:], axis=0), (-1,4)), axis= 0)
                cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[7:8, :2], cur_coord[8:9, 2:], axis=0), (-1,4)), axis= 0)
                cur_coord = np.append( cur_coord, np.reshape([ cur_coord[8, 0], cur_coord[7, 1], cur_coord[8, 2], cur_coord[8, 3]], (-1,4)), axis= 0)
                cur_coord = np.append( cur_coord, np.reshape(np.append(cur_coord[0:1, :2], cur_coord[8:9, 2:], axis=0), (-1,4)), axis= 0)

                coords = np.append(coords[-2:], np.reshape(cur_coord, (-1,38,4)), axis=0)

                coord = np.median(coords, axis =0 )
                
                
                #디렉스에서 요청했던 1, 2 값 추가 39번 : 1, 40 번 : 2
                shoulder_m = [x+y//2 for x,y in zip(coord[11], coord[12])]
                gol_m = [x+y//2 for x,y in zip(coord[23], coord[24])]
                
                coord = np.append(coord, [shoulder_m], axis=0)
                coord = np.append(coord, [gol_m], axis=0)
                
                
                
                input_angles, input_stands = get_joint_angle(coord)
                
                if DJoints!= None and DJoints[0] in joints[cur_id] :
                    dindex = joints[cur_id].index(DJoints[0])
                    distance = np.array([get_distance( *coord[j, :3]) for j in DJoints])
                    input_angles[dindex] = distance
                else:
                    dindex = None
                #frame = cv2.resize(frame, dsize=(640, 480 ))
                inputs = []
                ins = []
                l_count = 0
                for i in range(len(input_angles)):
                    ins.append(input_angles[i])
                    if i == j_len_list[l_count]-1:
                        inputs.append(ins)
                        ins = []
                        l_count += 1
                        
                        
                cv2.rectangle(frame, (frame.shape[1]-100, 0), (frame.shape[1],40), (255,255,255), -1)
                p_text = f"phase : {cur_id}"
                cv2.putText(frame, p_text, (frame.shape[1]-100, 20), cv2.FONT_HERSHEY_SIMPLEX,0.6, (0, 0, 0), 2)
                
                cv2.rectangle(frame, (0, 0), (230, ((len(inputs[cur_id])-1)*30+85)), (255,255,255), -1)
                joint_title = "User Inputs"
                cv2.putText(frame, joint_title, (0, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 120, 105), 2)
                
                z = 75
                for i in range(len(inputs[cur_id])):
                    text = f"{joints[cur_id][i]} : {inputs[cur_id][i]:.0f}"
                    if dindex != None and i == dindex :
                        text = f"{joints[cur_id][i]} : {inputs[cur_id][i]:.2f}"
                    cv2.putText(frame, text, (10, z), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,0.7, (0, 0, 0), 2)
                    z = z + 30
                
                            

                cv2.imshow('frame', frame)
                

                key = cv2.waitKey(1) & 0xFF

                if key == ord("q"):
                    break


    # ================
    if move_number+1 in MANY_KP:
        view_2()
    else:
        view_1()
    cv2.destroyAllWindows()
    

just_run(30)

<< 30 . 서서 사이드 스트레칭 >>

원본 스트레칭 영상 각도
방향1
[0, 38, 39] : 140 °
[16, 12, 24] : 169 °
[15, 11, 23] : 130 °
[16, 12, 11] : 111 °
[15, 11, 12] : 46 °
[11, 12] : 24 °
[23, 24] : 1 °
[24, 23, 27] : 61 °
[23, 24, 28] : 83 °
[23, 24, 26] : 88 °
[24, 23, 25] : 59 °
[6, 10, 12] : 80 °
[7, 9, 11] : 128 °


방향2
[0, 38, 39] : 127 °
[16, 12, 24] : 130 °
[15, 11, 23] : 163 °
[16, 12, 11] : 52 °
[15, 11, 12] : 109 °
[11, 12] : 27 °
[23, 24] : 3 °
[24, 23, 27] : 76 °
[23, 24, 28] : 62 °
[23, 24, 26] : 65 °
[24, 23, 25] : 80 °
[6, 10, 12] : 91 °
[7, 9, 11] : 98 °


